This Python script scrapes complaint data from the Better Business Bureau website for Securus Technologies, LLC, one of the largest private vendors in U.S. prison communications. Using Selenium and BeautifulSoup, it automates page navigation, expands hidden complaint details, and extracts key information — date, type, status, and complaint body — from all available pages.

The resulting dataset offers a window into the struggles families face trying to communicate with incarcerated loved ones and navigate the costly, often unreliable systems that mediate their connection to the outside world.

In [7]:
# First things first, let's download Selenium
# We use Selenium because the BBB complaints page is dynamically generated with JavaScript, 
# meaning the complaint data doesn’t appear in the static HTML source. Selenium loads the full
# webpage in a real browser session, allowing us to scrape the rendered complaint data accurately.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 38.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0
Note: you may need to restart the kernel to use updated packages.


In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time, random, traceback

## Now let's set up our coding infrastructure 

BASE_URL = "https://www.bbb.org/us/tx/carrollton/profile/government-contractors/securus-technologies-llc-0875-41000098/complaints?page="
HEADLESS = True
TOTAL_PAGES = 32
OUTPUT_FILE = "securus_bbb_complaints_complete.csv"

def create_driver():
## Launches a fresh Chrome driver with the right options."""
    options = Options()
    if HEADLESS:
        options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.add_argument("start-maximized")
    options.add_argument(
        "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    )
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver = create_driver()
all_complaints = []

## Now let's create a Scraper Loop 

try:
    for page in range(1, TOTAL_PAGES + 1):
        try:
            url = f"{BASE_URL}{page}"
            print(f"Scraping page {page}/{TOTAL_PAGES}: {url}")
            driver.get(url)

## Wait for complaint cards to load
            wait = WebDriverWait(driver, 30)
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.card.bpr-complaint-grid")))
            time.sleep(random.uniform(2, 4))

## Expand “More info” in complaint statuses
            detail_buttons = driver.find_elements(By.CSS_SELECTOR, ".bpr-complaint-status details summary")
            for btn in detail_buttons:
                try:
                    driver.execute_script("arguments[0].click();", btn)
                    time.sleep(0.1)
                except:
                    continue

## Let's parse the HTML with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.select("li.card.bpr-complaint-grid")

            for li in cards:
                date = li.select_one(".bpr-complaint-date span")
                ctype = li.select_one(".bpr-complaint-type span")
                body = li.select_one(".bpr-complaint-body div")

                # Only take the summary text for status
                status_summary = li.select_one(".bpr-complaint-status summary")
                status = status_summary.get_text(strip=True) if status_summary else None

                all_complaints.append({
                    "date": date.get_text(strip=True) if date else None,
                    "type": ctype.get_text(strip=True) if ctype else None,
                    "status": status,
                    "body": body.get_text(" ", strip=True) if body else None
                })

            print(f"Found {len(cards)} complaints on this page ({len(all_complaints)} total).")

## Save progress every 5 pages
            if page % 5 == 0:
                pd.DataFrame(all_complaints).to_csv(OUTPUT_FILE, index=False)
                print(f"Progress saved after page {page}.")

## Let's create randomized human-like pauses
            time.sleep(random.uniform(3, 7))
            if page % 10 == 0:
                time.sleep(random.uniform(10, 20))

        except Exception as e:
            print(f"Error on page {page}: {e}")
            print("Reinitializing browser...")
            driver.quit()
            driver = create_driver()
            time.sleep(5)
            continue  # retry next page

## Create our Data Frame and export for use
    df = pd.DataFrame(all_complaints)
    df.to_csv(OUTPUT_FILE, index=False)
    print(f"\n Scraping complete! {len(df)} total complaints saved to {OUTPUT_FILE}")

except Exception:
    print("Fatal error:\n")
    print(traceback.format_exc())

finally:
    driver.quit()

Scraping page 1/32: https://www.bbb.org/us/tx/carrollton/profile/government-contractors/securus-technologies-llc-0875-41000098/complaints?page=1
Found 10 complaints on this page (10 total).
Scraping page 2/32: https://www.bbb.org/us/tx/carrollton/profile/government-contractors/securus-technologies-llc-0875-41000098/complaints?page=2
Found 10 complaints on this page (20 total).
Scraping page 3/32: https://www.bbb.org/us/tx/carrollton/profile/government-contractors/securus-technologies-llc-0875-41000098/complaints?page=3
Found 10 complaints on this page (30 total).
Scraping page 4/32: https://www.bbb.org/us/tx/carrollton/profile/government-contractors/securus-technologies-llc-0875-41000098/complaints?page=4
Found 10 complaints on this page (40 total).
Scraping page 5/32: https://www.bbb.org/us/tx/carrollton/profile/government-contractors/securus-technologies-llc-0875-41000098/complaints?page=5
Found 10 complaints on this page (50 total).
Progress saved after page 5.
Scraping page 6/32: h

In [13]:
## Now, let's display the first 10 rows in a readable format
print("\nPreview of scraped data:\n")
display(df.head(10))


Preview of scraped data:



,date,type,status,body
0,09/22/2025,Service or Repair Issues,UnansweredMore info,My love one calls me and the recording keeps g...
1,09/19/2025,Billing Issues,UnansweredMore info,06-17-2025 & 06-19-2025 I made 2 purchases 1 f...
2,09/18/2025,Service or Repair Issues,UnansweredMore info,"Endless issue, with message/pictures not being..."
3,09/15/2025,Product Issues,UnansweredMore info,I am writing to address the problem I am havin...
4,09/12/2025,Service or Repair Issues,UnansweredMore info,"I, **** ******* #*******, an inmate of Farming..."
5,09/09/2025,Service or Repair Issues,UnansweredMore info,Securus had a DATA BREACH affecting the millio...
6,09/07/2025,Service or Repair Issues,UnansweredMore info,So recently I started getting these spam calls...
7,09/05/2025,Customer Service Issues,UnansweredMore info,Securus Technologies ******* refused to provid...
8,08/30/2025,Customer Service Issues,UnansweredMore info,This business has absolutely no Customer Suppo...
9,08/27/2025,Service or Repair Issues,UnansweredMore info,There is always a problem trying to log in to ...
